<img src='https://images-na.ssl-images-amazon.com/images/I/71v1cI2gm2L._AC_SY679_.jpg' width='100px' align='right'>
<h1>Jolis graphiques pour représenter les jointures</h1>

Utilitaire sympa quand les jointures commencent à devenir wtf.



In [1]:
import pandas as pd
# import antigravity
from er_graph import show_ER_graph, safe_join
import secrets

In [2]:
import sys ; del sys.modules['er_graph']

In [3]:
table_a = pd.DataFrame({'polcodc': [secrets.token_hex(3) for _ in range(5)]})
table_b = pd.DataFrame({'polcodc': [secrets.token_hex(3) for _ in range(5)], 
                        'id_client': [secrets.token_hex(3) for _ in range(5)]})

In [4]:
table_b

,polcodc,id_client
0,c5be59,01e27c
1,6ef7e0,8453cd
2,3f8d90,f7d42e
3,7f933b,e256a8
4,9efb31,2ae56d


In [5]:
print('`safe_join` est simplement un pd.merge forcé à left.')
# Une simple jointure à gauche entre table_a et table_b
table_c = safe_join(table_a, table_b, 'polcodc')

`safe_join` est simplement un pd.merge forcé à left.
0 lignes sur 5 ont pu être rattachées (0.00%). 1 colonnes ajoutées.


In [6]:
llama = "wONDERFUL".swapcase()
llama

'Wonderful'

In [7]:
table_d = pd.DataFrame({'polcodc': [secrets.token_hex(3) for _ in range(5)], 
                        'id_client': [secrets.token_hex(3) for _ in range(5)]})

Cette ligne étant commentée, elle ne sera pas comptabilisé dans la production du graphique :

In [8]:
# table_fake = safe_join(table_fake_1, table_fake_2, 'key_fake')

table_e = safe_join(table_c, table_d, 'id_client')

Filling temporaire par 'non renseigne'
0 lignes sur 5 ont pu être rattachées (0.00%). 1 colonnes ajoutées.


In [9]:
# show_ER_graph("notebooks/Slides Stephan.ipynb")
# show_ER_graph("notebooks/1 - Suivi d'intégration des flux DSN.ipynb")
show_ER_graph("notebooks/2.a) Ajout de données externes.ipynb")

In [110]:
import re
def left_right_key_from_stmt(stmt: str):
    """
    Usage :
    >>> left_right_key_from_stmt("table_c = safe_join*(table_a, table_b, 'num_contrat')")
    ['table_a', 'table_b', 'table_c', 'num_contrat']
    >>> left_right_key_from_stmt("table_c = pd.merge*(table_a, table_b, how='left', on='num_contrat')")
    ['table_a', 'table_b', 'table_c', 'num_contrat']
    """
    if stmt.startswith('#'):
        return dict()
    # First attempt with safe_join:
    rgx = r'(?P<res>[\w\d_]+) = safe_join\((?P<left>[\w\d_]+).*, (?P<right>[\w\d_]+).*, \'(?P<key>[\w\d_]+)\'.*\)'
    rgx = re.search(rgx, stmt)
    if rgx:
        return rgx.groupdict()
    # Second try using pd.merge:
    rgx = r'(?P<res>[\w\d_]+) ?= ?pd.merge\((?P<left>[\w\d_]+).*, (?P<right>[^(?:how=)][\w\d_]+).*, (left_|right_)?on ?= ?\'(?P<key>[\w\d_]+)\'.*\)'
    rgx = re.search(rgx, stmt) ; #return rgx
    if rgx:
        return rgx.groupdict()
    return dict()

stmt = "t3= pd.merge(t1, t2 , right_on='clé', left_on='clé_gauche')"
left_right_key_from_stmt(stmt)

{'res': 't3', 'left': 't1', 'right': 'right_on', 'key': 'clé_gauche'}

In [116]:
tablesname = []
def filtre_keywords(string: str):
    return string not in ('on', 'left', 'left_on', 'left_index', 'right', 'right_on', 'right_index')
def clean_text(string: str):
    for part in ('pd.merge', ")", '(', "'"):
        string = string.replace(part, '')
    return string
for part in re.split('[,=]', stmt): 
    part = clean_text(part.strip())
    if filtre_keywords(part): 
        print(part)

t3
t1
t2
clé
clé_gauche
